# **<p style="text-align: center;">Aprendizaje por refuerzo - Ingeniería en Inteligencia Artificial</p>**
## **<p style="text-align: center;">Trabajo práctico Final - SAC</p>**

#### <p style="text-align: center;">Pettinari Fausto, Schuemer Ignacio, Torres Santiago </p>
#### <p style="text-align: center;">Profesores: Claudio Pose, Gabriel Torre, Nicolás Romero, Tomás Chimenti</p>

# **One DPoint Mass Reach Enviroment - *Entorno de Prueba***

In [1]:
import gymnasium as gym
from envs import OneDPointMassReachEnv

In [2]:
gym.register(
    id="gymnasium_env/OneDPointMassReachEnv-v0",
    entry_point=OneDPointMassReachEnv
)
env = gym.make("gymnasium_env/OneDPointMassReachEnv-v0")

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import sys
import os
sys.path.append(os.path.join('..'))
from src.random_agent import random_agent_loop

#### **Logging & Debugging (Random Agent)**

In [ ]:
seed = 42
writer = SummaryWriter(f"../runs/OneDPointMassReachEnv")
num_episodes = 5000
random_agent_loop(env, num_episodes, writer, seed)


2025-11-20 20:07:18.000574: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-20 20:07:18.039849: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 20:07:19.067143: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Episode 0: avg batch reward = -0.500
Episode 10: avg batch reward = -0.500
Episode 20: avg batch reward = -0.500
Episode 30: avg batch reward = -0.500
Episode 40: avg batch reward = -0.500
Episode 50: avg batch reward = -0.393
Episode 60: avg batch reward = -0.500
Episode 70: avg batch reward = -0.500
Episode 80: avg batch reward = -0.500
Episode 90: avg batch reward = -0.397
Episode 100: avg batch reward = -0.500
Episode 110: avg batch reward = -0.500
Episode 120: avg batch reward = -0.500
Episode 130: avg batch reward = -0.500
Episode 140: avg batch reward = -0.393
Episode 150: avg batch reward = -0.500
Episode 160: avg batch reward = -0.500
Episode 170: avg batch reward = -0.500
Episode 180: avg batch reward = -0.500
Episode 190: avg batch reward = -0.500
Episode 200: avg batch reward = -0.500
Episode 210: avg batch reward = -0.396
Episode 220: avg batch reward = -0.500
Episode 230: avg batch reward = -0.500
Episode 240: avg batch reward = -0.500
Episode 250: avg batch reward = -0.5

## **SAC Agent**

In [ ]:
from src.SAC import SAC
from hyperparameters import SACConfig